# ✈️ Aviation Accidents - Exploratory Data Analysis

**Author**: [Il Tuo Nome]  
**Dataset**: Aviation Safety Network (ASN) - 23,967 incidents (1919-2023)  
**Objective**: Comprehensive EDA with OOP architecture, temporal/categorical/geospatial analysis

In [ ]:
# === IMPORTS ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import re
import warnings
from typing import Optional, Tuple, List, Dict
from dataclasses import dataclass
import os

warnings.filterwarnings('ignore')

# === VISUAL SETTINGS ===
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
PALETTE = 'rocket'

print('✅ Libraries loaded successfully')

In [ ]:
# === DATA LOADING ===
def load_data():
    paths_to_try = [
        'aviation_accidents.csv',
        '../data/aviation_accidents.csv',
        '/content/aviation_accidents.csv',
        '/content/drive/MyDrive/aviation_accidents.csv'
    ]
    for path in paths_to_try:
        if os.path.exists(path):
            print(f'📂 Loading from: {path}')
            return pd.read_csv(path)
    try:
        from google.colab import files
        print('⬆️ Please upload aviation_accidents.csv')
        uploaded = files.upload()
        return pd.read_csv(list(uploaded.keys())[0])
    except:
        raise FileNotFoundError('Dataset not found!')

df_raw = load_data()
print(f'📊 Shape: {df_raw.shape[0]:,} rows × {df_raw.shape[1]} columns')

In [ ]:
# === CATEGORY MAPPING ===
CAT_MAPPING = {
    'A1': 'Accident - Hull Loss', 'A2': 'Accident - Repairable',
    'C1': 'Criminal - Fatal', 'C2': 'Criminal - Non-Fatal',
    'H1': 'Hijacking - Fatal', 'H2': 'Hijacking - Non-Fatal',
    'O1': 'Other - Hull Loss', 'O2': 'Other - Repairable',
    'I1': 'Incident - Fatal', 'I2': 'Incident - Non-Fatal', 'U1': 'Unknown'
}

In [ ]:
@dataclass
class CleaningStats:
    rows_original: int
    rows_after: int
    dates_parsed: int
    dates_failed: int
    fatalities_imputed: int
    outliers_removed: int

class UniversalDataEngine:
    def __init__(self, df):
        self.df_raw = df.copy()
        self.df = None
        self.cleaning_stats = None
        self._cat_mapping = CAT_MAPPING
    
    def _parse_date(self, date_str):
        if pd.isna(date_str) or date_str in ['date unk.', 'unknown', '']:
            return pd.NaT
        for fmt in ['%d-%b-%Y', '%Y-%m-%d', '%m/%d/%Y', '%d/%m/%Y']:
            try:
                return pd.to_datetime(date_str, format=fmt)
            except:
                continue
        return pd.NaT
    
    def _extract_fatalities(self, fat_str):
        if pd.isna(fat_str) or str(fat_str).strip() == '':
            return (0, 0, True)
        fat_str = str(fat_str).strip()
        if '+' in fat_str:
            parts = fat_str.split('+')
            try:
                return (int(parts[0].strip()), int(parts[1].strip()) if len(parts) > 1 else 0, False)
            except:
                return (0, 0, True)
        try:
            return (int(fat_str), 0, False)
        except:
            return (0, 0, True)
    
    def _remove_outliers_iqr(self, series, k=1.5):
        q1, q3 = series.quantile(0.25), series.quantile(0.75)
        iqr = q3 - q1
        return (series < q1 - k*iqr) | (series > q3 + k*iqr)
    
    def clean(self, remove_outliers=False):
        print('🧹 Starting cleaning...')
        df = self.df_raw.copy()
        rows_original = len(df)
        
        for col in ['type', 'registration', 'operator', 'location', 'country']:
            df[col] = df[col].astype(str).str.strip().str.lower()
            df[col] = df[col].replace(['nan', 'none', '?', 'unknown', '', 'unknown country'], np.nan)
        
        print('  📅 Parsing dates...')
        df['date_parsed'] = df['date'].apply(self._parse_date)
        df['year_clean'] = df['date_parsed'].dt.year
        df['month'] = df['date_parsed'].dt.month
        df['day_of_week'] = df['date_parsed'].dt.dayofweek
        df['decade'] = (df['year_clean'] // 10 * 10).astype('Int64')
        
        print('  💀 Extracting fatalities...')
        fat = df['fatalities'].apply(self._extract_fatalities)
        df['fatalities_aboard'] = fat.apply(lambda x: x[0])
        df['fatalities_ground'] = fat.apply(lambda x: x[1])
        df['fatalities_total'] = df['fatalities_aboard'] + df['fatalities_ground']
        df['is_imputed'] = fat.apply(lambda x: x[2])
        
        df['cat_description'] = df['cat'].map(self._cat_mapping)
        df['is_fatal'] = df['fatalities_total'] > 0
        df['is_hull_loss'] = df['cat'].isin(['A1', 'O1', 'C1', 'H1'])
        df['manufacturer'] = df['type'].str.split().str[0]
        
        df['is_outlier'] = self._remove_outliers_iqr(df['fatalities_total']) if remove_outliers else False
        
        self.df = df
        self.cleaning_stats = CleaningStats(
            rows_original, len(df), df['date_parsed'].notna().sum(),
            df['date_parsed'].isna().sum(), df['is_imputed'].sum(),
            df['is_outlier'].sum() if remove_outliers else 0
        )
        print(f'✅ Done! Dates parsed: {self.cleaning_stats.dates_parsed:,}')
        return self
    
    def eda_temporal(self, show_911=True):
        df = self.df[self.df['date_parsed'].notna()].copy()
        print('\n' + '='*60 + '\n📈 TEMPORAL ANALYSIS\n' + '='*60)
        
        yearly = df.groupby('year_clean').agg(
            incidents=('date_parsed', 'count'), fatalities=('fatalities_total', 'sum')
        ).reset_index()
        
        fig, ax1 = plt.subplots(figsize=(14, 6))
        c1, c2 = sns.color_palette(PALETTE, 10)[2], sns.color_palette(PALETTE, 10)[7]
        ax1.fill_between(yearly['year_clean'], yearly['incidents'], alpha=0.3, color=c1)
        ax1.plot(yearly['year_clean'], yearly['incidents'], color=c1, lw=2)
        ax1.set_xlabel('Year'); ax1.set_ylabel('Incidents', color=c1)
        ax2 = ax1.twinx()
        ax2.plot(yearly['year_clean'], yearly['fatalities'], color=c2, lw=2, ls='--')
        ax2.set_ylabel('Fatalities', color=c2)
        if show_911:
            ax2.axvline(2001, color='red', ls=':', alpha=0.7)
        plt.title('Aviation Incidents and Fatalities Over Time', fontsize=14, fontweight='bold')
        plt.tight_layout(); plt.show()
        return {'yearly': yearly}
    
    def eda_categorical(self):
        df = self.df.copy()
        print('\n' + '='*60 + '\n📊 CATEGORICAL ANALYSIS\n' + '='*60)
        
        cat_stats = df.groupby(['cat', 'cat_description']).agg(
            incidents=('date', 'count'), fatalities=('fatalities_total', 'sum')
        ).reset_index().sort_values('incidents', ascending=False)
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        sns.barplot(data=cat_stats, y='cat_description', x='incidents', palette=PALETTE, ax=axes[0])
        axes[0].set_title('Incidents by Category', fontweight='bold')
        sns.barplot(data=cat_stats, y='cat_description', x='fatalities', palette=PALETTE, ax=axes[1])
        axes[1].set_title('Fatalities by Category', fontweight='bold')
        plt.tight_layout(); plt.show()
        
        manuf = df[df['manufacturer'].notna()].groupby('manufacturer').agg(
            incidents=('date', 'count'), fatalities=('fatalities_total', 'sum')
        ).reset_index().nlargest(15, 'incidents')
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        sns.barplot(data=manuf, y='manufacturer', x='incidents', palette=PALETTE, ax=axes[0])
        axes[0].set_title('Top 15 Manufacturers by Incidents', fontweight='bold')
        sns.barplot(data=manuf.nlargest(15, 'fatalities'), y='manufacturer', x='fatalities', palette=PALETTE, ax=axes[1])
        axes[1].set_title('Top 15 Manufacturers by Fatalities', fontweight='bold')
        plt.tight_layout(); plt.show()
        return {'categories': cat_stats}
    
    def eda_geospatial(self):
        df = self.df[self.df['country'].notna()].copy()
        print('\n' + '='*60 + '\n🗺️ GEOSPATIAL ANALYSIS\n' + '='*60)
        
        # MAPPING ESTESO E NORMALIZZATO
        name_map = {
            'usa': 'United States',
            'united states': 'United States',
            'u.s.': 'United States',
            'u.s. virgin isl.': 'United States',
            'puerto rico': 'United States',
            'guam': 'United States',
            'u.k.': 'United Kingdom',
            'united kingdom': 'United Kingdom',
            'england': 'United Kingdom',
            'scotland': 'United Kingdom',
            'russia': 'Russia',
            'ussr': 'Russia',
            'russian federation': 'Russia',
            'd.r. congo': 'Democratic Republic of the Congo',
            'democratic republic of the congo': 'Democratic Republic of the Congo',
            'congo': 'Republic of the Congo',
            'china (prc)': 'China',
            'prc': 'China',
            'korea, south': 'South Korea',
            'korea, north': 'North Korea',
            'taiwan': 'Taiwan',
            'republic of china': 'Taiwan',
        }
        
        country_stats = df.groupby('country').agg(
            incidents=('date', 'count'),
            fatalities=('fatalities_total', 'sum')
        ).reset_index()
        
        country_stats['country_normalized'] = country_stats['country'].map(
            lambda x: name_map.get(x, x.title()) if pd.notna(x) else x
        )
        
        # RI-AGGREGA dopo normalizzazione per unire territori
        country_stats_final = country_stats.groupby('country_normalized').agg(
            incidents=('incidents', 'sum'),
            fatalities=('fatalities', 'sum')
        ).reset_index().rename(columns={'country_normalized': 'country_plotly'})
        
        # DEBUG
        usa_stats = country_stats_final[country_stats_final['country_plotly'] == 'United States']
        print(f"\n🔍 DEBUG - USA Stats:\n{usa_stats}\n")
        
        # MAPPA 1: Incidents (Viridis)
        fig = px.choropleth(
            country_stats_final,
            locations='country_plotly',
            locationmode='country names',
            color='incidents',
            hover_name='country_plotly',
            hover_data={'incidents': ':,', 'fatalities': ':,'},
            color_continuous_scale='Viridis',
            title='<b>Aviation Incidents by Country</b>'
        )
        fig.update_layout(geo=dict(showframe=False, projection_type='natural earth'))
        fig.show()
        
        # MAPPA 2: Fatalities (Inferno)
        fig2 = px.choropleth(
            country_stats_final,
            locations='country_plotly',
            locationmode='country names',
            color='fatalities',
            hover_name='country_plotly',
            hover_data={'incidents': ':,', 'fatalities': ':,'},
            color_continuous_scale='Inferno',
            title='<b>Aviation Fatalities by Country</b>'
        )
        fig2.update_layout(geo=dict(showframe=False, projection_type='natural earth'))
        fig2.show()
        
        # BAR CHARTS
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        top_incidents = country_stats_final.nlargest(20, 'incidents')
        sns.barplot(data=top_incidents, y='country_plotly', x='incidents', palette=PALETTE, ax=axes[0])
        axes[0].set_title('Top 20 Countries by Incidents', fontweight='bold')
        axes[0].set_xlabel('Number of Incidents'); axes[0].set_ylabel('')
        
        top_fatalities = country_stats_final.nlargest(20, 'fatalities')
        sns.barplot(data=top_fatalities, y='country_plotly', x='fatalities', palette=PALETTE, ax=axes[1])
        axes[1].set_title('Top 20 Countries by Fatalities', fontweight='bold')
        axes[1].set_xlabel('Total Fatalities'); axes[1].set_ylabel('')
        plt.tight_layout(); plt.show()
        
        return {'country_stats': country_stats_final}

print('✅ UniversalDataEngine defined')

In [ ]:
# === RUN ANALYSIS ===
engine = UniversalDataEngine(df_raw)
engine.clean(remove_outliers=True)

In [ ]:
temporal_results = engine.eda_temporal(show_911=True)

In [ ]:
categorical_results = engine.eda_categorical()

In [ ]:
geo_results = engine.eda_geospatial()

## 🔬 Deep Dive: 9/11 Impact & Aircraft Lethality

In [ ]:
# === 9/11 ANALYSIS ===
print('='*60 + '\n🔬 9/11 IMPACT ANALYSIS\n' + '='*60)
df = engine.df
sept_11 = df[df['date'] == '11-SEP-2001']
print(f'\n📅 Events on 9/11: {len(sept_11)}')
print(sept_11[['type', 'operator', 'fatalities_aboard', 'fatalities_ground', 'fatalities_total', 'location']])

fat_2001 = df[df['year_clean']==2001]['fatalities_total'].sum()
fat_911 = sept_11['fatalities_total'].sum()
print(f'\n📊 2001 total: {fat_2001:,} | 9/11 only: {fat_911:,} ({fat_911/fat_2001*100:.1f}%)')

In [ ]:
# === AIRCRAFT LETHALITY ===
print('\n✈️ AIRCRAFT LETHALITY')
type_stats = df.groupby('type').agg(
    incidents=('date', 'count'), fatalities=('fatalities_total', 'sum'),
    avg_fat=('fatalities_total', 'mean')
).reset_index()
type_stats = type_stats[type_stats['incidents'] >= 20].sort_values('avg_fat', ascending=False)

fig, axes = plt.subplots(1, 2, figsize=(14, 7))
sns.barplot(data=type_stats.head(20), y='type', x='avg_fat', palette=PALETTE, ax=axes[0])
axes[0].set_title('Top 20 Most Lethal Aircraft (Avg)', fontweight='bold')
sns.barplot(data=type_stats.nlargest(20, 'fatalities'), y='type', x='fatalities', palette=PALETTE, ax=axes[1])
axes[1].set_title('Top 20 by Total Fatalities', fontweight='bold')
plt.tight_layout(); plt.show()

In [ ]:
# === OPERATOR LETHALITY ===
op_stats = df[df['operator'].notna()].groupby('operator').agg(
    incidents=('date', 'count'), fatalities=('fatalities_total', 'sum'),
    avg_fat=('fatalities_total', 'mean')
).reset_index()
op_stats = op_stats[op_stats['incidents'] >= 15]

fig, axes = plt.subplots(1, 2, figsize=(14, 7))
sns.barplot(data=op_stats.nlargest(20, 'avg_fat'), y='operator', x='avg_fat', palette=PALETTE, ax=axes[0])
axes[0].set_title('Top 20 Operators by Avg Fatalities', fontweight='bold')
sns.barplot(data=op_stats.nlargest(20, 'fatalities'), y='operator', x='fatalities', palette=PALETTE, ax=axes[1])
axes[1].set_title('Top 20 Operators by Total Fatalities', fontweight='bold')
plt.tight_layout(); plt.show()

In [ ]:
# === BUBBLE CHART ===
manuf_stats = df[df['manufacturer'].notna()].groupby('manufacturer').agg(
    incidents=('date', 'count'), fatalities=('fatalities_total', 'sum'),
    fatal_rate=('is_fatal', 'mean')
).reset_index().nlargest(30, 'incidents')

fig = px.scatter(manuf_stats, x='incidents', y='fatal_rate', size='fatalities', color='fatalities',
    hover_name='manufacturer', color_continuous_scale='inferno',
    title='<b>Manufacturer Risk Profile</b><br><sup>Bubble size = Fatalities</sup>',
    labels={'incidents': 'Incidents', 'fatal_rate': 'Fatal Rate', 'fatalities': 'Fatalities'})
fig.update_layout(yaxis_tickformat='.0%')
fig.show()

In [ ]:
# === SUMMARY ===
print('='*60 + '\n📊 FINAL SUMMARY\n' + '='*60)
print(f'''\nDataset: {len(df):,} records ({df["year_clean"].min():.0f}-{df["year_clean"].max():.0f})
Total fatalities: {df["fatalities_total"].sum():,}
Fatal incidents: {df["is_fatal"].sum():,} ({df["is_fatal"].mean()*100:.1f}%)
9/11 fatalities: {df[df["date"]=="11-SEP-2001"]["fatalities_total"].sum():,}''')